In [ ]:
# STL
import os
import json
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
# 3rd Party
import torch
from copy import deepcopy
from transformers import  BertTokenizerFast
# Local
from gatbert.data import parse_graph_tsv
from gatbert.constants import NodeType

/home/ethanlmines/blue_dir/conda_envs/gatbert/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pretrained_model_name = 'bert-base-cased'
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name)

In [3]:
sample = next(parse_graph_tsv('scrap.tsv'))

In [5]:
tokenized_text = tokenizer(text=sample.target, text_pair=sample.context, is_split_into_words=True, return_offsets_mapping=True, return_tensors='pt')
tokenized_kb = tokenizer(text=sample.kb, is_split_into_words=True, return_offsets_mapping=True, return_tensors='pt')

Token indices sequence length is longer than the specified maximum sequence length for this model (2817 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
pool_indices = []

In [29]:
combined_offsets = torch.concatenate([tokenized_text['offset_mapping'].squeeze(), tokenized_text['offset_mapping'].squeeze()])

In [ ]:
from collections import defaultdict

# (node_index, subword_index)
pool_inds = []

expand_list = defaultdict(list)

new_nodes_index = 0
orig_nodes_index = 0

token_offset_mapping = tokenized_text['offset_mapping'].squeeze()

# Handle splitting of token nodes into subword nodes
for (subword_index, (start, end)) in enumerate(token_offset_mapping):
    if end == start:
        # Special character; skip over
        pool_inds.append((new_nodes_index, subword_index))
    else:
        if start == 0:
            orig_nodes_index += 1
        pool_inds.append((new_nodes_index, subword_index))
        expand_list[orig_nodes_index].append(subword_index)
    new_nodes_index += 1

# Get this working next
n_subwords = subword_index + 1
for (subword_index, (start, end)) in enumerate(tokenized_kb['offset_mapping'].squeeze()):
    if end == start:
        continue
    if start == 0:
        pass
    pool_inds.append((new_nodes_index, subword_index))
    new_nodes_index += 1


# Handle pool indices for 

In [ ]:
trans_edges = []
for edge in sample.edges:
    head_inds = expand
    tail_inds = []

    if edge_cop.head_graph_index == NodeType.TOKEN:
        for subword_ind in expand_list[edge.head_node_index]:
            edge_cop = deepcopy(edge)

            pass


In [5]:

split2orig = []
cur_index = 0
for (i, (offset, length)) in enumerate(results['offset_mapping']):
    if length == 0:
        cur_index += 1
    elif offset == 0:
        cur_index += 1
        split2orig.append(cur_index)
    else:
        split2orig.append(cur_index)

In [6]:
tokenizer.convert_ids_to_tokens(results['input_ids'])

['[CLS]',
 'Pakistan',
 'government',
 '[SEP]',
 'Pakistan',
 'government',
 'Re',
 '##quest',
 'more',
 'help',
 'from',
 'the',
 'world',
 'community',
 '.',
 'after',
 'facing',
 'the',
 'flood',
 'The',
 'largest',
 'in',
 'history',
 'causing',
 'heavy',
 'damage',
 'across',
 'the',
 'country',
 '1',
 ',',
 '129',
 'people',
 'have',
 'died',
 ',',
 '5',
 ',',
 '000',
 'people',
 'have',
 'been',
 'injured',
 ',',
 'and',
 '50',
 'million',
 'people',
 'have',
 'been',
 'evacuated',
 '.',
 'C',
 '##r',
 ':',
 'T',
 '##N',
 '##N',
 'World',
 'News',
 'The',
 'following',
 'media',
 'includes',
 'potentially',
 'sensitive',
 'content',
 '.',
 'Change',
 'settings',
 'View',
 '[SEP]']

In [20]:
cn_names = [uri.split('/')[3] for uri in sample.external][-5:]
res2 = tokenizer(text=cn_names, is_split_into_words=True, return_offsets_mapping=True)
tokenizer.convert_ids_to_tokens(res2['input_ids'] ), res2['offset_mapping']

(['[CLS]',
  'higher',
  '_',
  'number',
  '_',
  'than',
  '_',
  '0',
  'b',
  '##ool',
  '##ean',
  'ps',
  '_',
  '1',
  'internet',
  'b',
  '##ool',
  '##ean',
  '[SEP]'],
 [(0, 0),
  (0, 6),
  (6, 7),
  (7, 13),
  (13, 14),
  (14, 18),
  (18, 19),
  (19, 20),
  (0, 1),
  (1, 4),
  (4, 7),
  (0, 2),
  (2, 3),
  (3, 4),
  (0, 8),
  (0, 1),
  (1, 4),
  (4, 7),
  (0, 0)])

In [19]:
res2.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

In [13]:
sample.external

['/c/en/pakistan',
 '/c/en/government',
 '/c/en/request',
 '/c/en/more',
 '/c/en/help',
 '/c/en/from',
 '/c/en/the',
 '/c/en/world',
 '/c/en/community',
 '/c/en/after',
 '/c/en/facing',
 '/c/en/flood',
 '/c/en/largest',
 '/c/en/in',
 '/c/en/history',
 '/c/en/causing',
 '/c/en/heavy',
 '/c/en/damage',
 '/c/en/across',
 '/c/en/country',
 '/c/en/1',
 '/c/en/people',
 '/c/en/have',
 '/c/en/died',
 '/c/en/5',
 '/c/en/000',
 '/c/en/been',
 '/c/en/injured',
 '/c/en/and',
 '/c/en/50',
 '/c/en/million',
 '/c/en/evacuated',
 '/c/en/cr',
 '/c/en/tnn',
 '/c/en/news',
 '/c/en/following',
 '/c/en/media',
 '/c/en/includes',
 '/c/en/potentially',
 '/c/en/sensitive',
 '/c/en/content',
 '/c/en/change',
 '/c/en/settings',
 '/c/en/view',
 '/c/en/drop_tower/n',
 '/c/en/vacuum_desiccator/n',
 '/c/en/unevacuated',
 '/c/en/vacuum_tube/n',
 '/c/en/thermionic_valve/n',
 '/c/en/electron_tube/n',
 '/c/en/aneroid_barometer/n',
 '/c/en/evacuable/a',
 '/c/en/evacuee/n',
 '/c/en/evacuate/v',
 '/c/en/5ever',
 '/c/en/a